In [1]:
import tushare as ts
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import baostock as bao
from baostock.data.resultset import ResultData
import baostock.common.contants as bao_consts
from pandas import Timestamp
from sz.stock_data.toolbox.data_provider import ts_code
from datetime import date, datetime, timedelta

ts.set_token('f96b1eeee9c8fddd357f2299cdedc1c88b2bb2a30ae1f772cf810dea')
ts_api = ts.pro_api()

blg = bao.login()
print('baostock login => error_code: %s error_msg: %s\n' %
      (blg.error_code, blg.error_msg))

from sz.stock_data.stock_data import StockData
StockData().setup(data_dir = '/Volumes/USBDATA/stock_data')


login success!
baostock login => error_code: 0 error_msg: success



/Users/kk/miniconda3/envs/quant/lib/python3.7/site-packages/tushare/util/upass.py:28: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated



In [2]:
from sz.stock_data.stocks.stock_daily import StockDaily
from sz.stock_data.toolbox.helper import need_update_by_trade_date

In [ ]:
stock = StockDaily(StockData().data_dir, stock_code = '300059.SZ')